In [2]:
import pandas as pd

### 1. Import Dataset

In [127]:
df_test = pd.read_excel('/Users/xuzhuxuan/Desktop/cia_test_control/test_cia.xlsx')
df_test = df_test.iloc[:,1:]
df_test

,id_mask,age,city,gender,set_126
0,0,30-34,广州市,女,"[15, 5]"
1,1,20-24,襄阳市,女,"[10, 1, 11, 13, 14, 4, 8]"
2,2,30-34,海口市,女,"[13, 4, 14]"
3,3,25-29,无锡市,女,"[1, 5, 12, 14, 11, 6, 4, 9, 2, 15]"
4,4,30-34,雅安市,女,"[15, 11, 12, 1, 5, 8, 4, 10, 13]"
...,...,...,...,...,...
32388,32388,25-29,北京市,女,"[15, 10, 3, 11, 2, 9, 14, 5, 7, 4]"
32389,32389,18-19,滁州市,女,"[3, 8, 10]"
32390,32390,18-19,贵阳市,女,"[11, 10, 9, 12, 15]"
32391,32391,20-24,安阳市,女,"[5, 11, 3, 7]"


In [128]:
df_test.dtypes

id_mask     int64
age        object
city       object
gender     object
set_126    object
dtype: object

In [129]:
df_control = pd.read_excel('/Users/xuzhuxuan/Desktop/cia_test_control/control_cia.xlsx')
df_control = df_control.iloc[:,1:]
df_control

,id_mask,age,city,gender,set_126
0,32393,25-29,洛阳市,女,"[13, 14, 12, 4, 6, 5, 2, 9]"
1,32394,25-29,揭阳市,男,"[1, 3, 13, 4, 2, 15, 10, 7, 9, 6]"
2,32395,20-24,北京市,女,"[6, 9, 8, 15, 2, 10]"
3,32396,45-49,广州市,女,"[15, 14, 10, 11, 9]"
4,32397,18-19,随州市,男,"[7, 15, 5, 2, 13, 4, 12, 1]"
...,...,...,...,...,...
275339,307732,18-19,西安市,女,"[4, 15, 1, 12]"
275340,307733,45-49,成都市,女,"[14, 5, 13, 6, 4, 9, 12, 11, 15]"
275341,307734,40-44,成都市,男,"[8, 7, 2, 4, 9, 12, 1, 5, 10]"
275342,307735,25-29,随州市,女,"[4, 5, 15]"


### 2. Data Preprocessing

In [130]:
# convert string to list
from ast import literal_eval
df_test['set_126'] = df_test['set_126'].apply(literal_eval)
df_control['set_126'] = df_control['set_126'].apply(literal_eval)

In [131]:
#Transform each element of a list-like to a row, replicating index values.
df_test = df_test.explode('set_126') 
df_control = df_control.explode('set_126') 
df_test

,id_mask,age,city,gender,set_126
0,0,30-34,广州市,女,15
0,0,30-34,广州市,女,5
1,1,20-24,襄阳市,女,10
1,1,20-24,襄阳市,女,1
1,1,20-24,襄阳市,女,11
...,...,...,...,...,...
32391,32391,20-24,安阳市,女,11
32391,32391,20-24,安阳市,女,3
32391,32391,20-24,安阳市,女,7
32392,32392,20-24,西安市,女,6


In [132]:
# reset dataframe index
df_test = df_test.reset_index(drop=True)
df_control = df_control.reset_index(drop=True)
df_test

,id_mask,age,city,gender,set_126
0,0,30-34,广州市,女,15
1,0,30-34,广州市,女,5
2,1,20-24,襄阳市,女,10
3,1,20-24,襄阳市,女,1
4,1,20-24,襄阳市,女,11
...,...,...,...,...,...
193909,32391,20-24,安阳市,女,11
193910,32391,20-24,安阳市,女,3
193911,32391,20-24,安阳市,女,7
193912,32392,20-24,西安市,女,6


In [133]:
# only focus on 1-5 in the set
df_test = df_test[df_test['set_126'] <= 5]
df_control = df_control[df_control['set_126'] <= 5] # only keep female in the control set
df_test

,id_mask,age,city,gender,set_126
1,0,30-34,广州市,女,5
3,1,20-24,襄阳市,女,1
7,1,20-24,襄阳市,女,4
10,2,30-34,海口市,女,4
12,3,25-29,无锡市,女,1
...,...,...,...,...,...
193899,32388,25-29,北京市,女,4
193900,32389,18-19,滁州市,女,3
193908,32391,20-24,安阳市,女,5
193910,32391,20-24,安阳市,女,3


In [136]:
age = df_test[df_test['id_mask'] == 32388]['age'].iloc[0]
city = df_test[df_test['id_mask'] == 32388]['city'].iloc[0]
gender = df_test[df_test['id_mask'] == 32388]['gender'].iloc[0]
st = set(df_test[df_test['id_mask'] == 32388]['set_126'])

In [139]:
df = df_control.loc[(df_control['age'] == age) & (df_control['city'] == city) & (df_control['gender'] == gender)].groupby('id_mask')['set_126'].apply(set)
import random
random.choice(df[df == st].index.tolist())

186851

In [140]:
df_test[df_test['id_mask'] == 32388]

,id_mask,age,city,gender,set_126
193892,32388,25-29,北京市,女,3
193894,32388,25-29,北京市,女,2
193897,32388,25-29,北京市,女,5
193899,32388,25-29,北京市,女,4


In [154]:
import random

test_list = df_test['id_mask'].unique().tolist()
match_list = []

for i in test_list:
    df1 = df_test[df_test['id_mask'] == i]
    age = df1['age'].iloc[0]
    city = df1['city'].iloc[0]
    gender = df1['gender'].iloc[0]
    st = set(df1['set_126'])
    df2 = df_control.loc[(df_control['age'] == age) &(df_control['city'] == city) & (df_control['gender'] == gender)].groupby('id_mask')['set_126'].apply(set)
    lst = df2[df2 == st].index.tolist()
    if lst:
        match_list.append(random.choice(lst))
    if not lst:
        match_list.append(None)

KeyboardInterrupt: 

In [125]:
df_test[df_test['age'] == '1'].index.tolist()

[]